Report number of cells:
- Cell types per sample, total, lowQ (if applicable)
- Beta subtypes per sample

In [41]:
import scanpy as sc
import pandas as pd
import gc
import numpy as np

In [62]:
path_data='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/combined/'
path_tables='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/tables/paper/'

In [6]:
# Get obs and usn of whole atlas and beta subset
temp=sc.read(path_data+'data_integrated_analysed.h5ad',backed='r')
obs=temp.obs.copy()
uns=temp.uns.copy()
del temp
obs_b=sc.read(path_data+'data_rawnorm_integrated_analysed_beta_v1s1_sfintegrated.h5ad',
              backed='r').obs.copy()
gc.collect()

372

In [58]:
# Pretty sample names
obs['study_parsed_design_sample']=obs.apply(
    lambda x: ' '.join([x.study_parsed,x.design,x.file]), axis=1)
obs_b['study_parsed_design_sample']=obs_b.apply(
    lambda x: ' '.join([x.study_parsed,x.design,x.file]), axis=1)

In [56]:
# Count cell types (total and per sample) and per cell type lowq and doublets
ct_count=pd.concat([
    obs['cell_type_integrated_v2_parsed'].value_counts().rename('total'),
    obs.query('low_q==False &  ~cell_type_integrated_v2_parsed.str.contains("\+")',
          engine='python')['cell_type_integrated_v2_parsed'].value_counts().\
          replace(0,np.nan).rename('low_quality'),
    obs.groupby('cell_type_integrated_v2_parsed').apply(lambda x:'+' in x.name
                                                       ).rename('doublet'),
    pd.crosstab(obs['cell_type_integrated_v2_parsed'],obs['study_parsed_design_sample'])
],axis=1)

In [57]:
ct_count

,total,low_quality,doublet,4m head_Fltp+ mouse2,4m head_Fltp- mouse1,4m tail_Fltp+ mouse4,4m tail_Fltp- mouse3,5wNOD IRE1alphabeta-/- SRR10985097,5wNOD IRE1alphabeta-/- SRR10985098,5wNOD IRE1alphafl/fl SRR10985099,...,embryonic E13.5 E13_5,embryonic E14.5 E14_5,embryonic E15.5 E15_5,mSTZ STZ G2,mSTZ STZ_GLP-1 G4,mSTZ STZ_GLP-1_estrogen G6,mSTZ STZ_GLP-1_estrogen+insulin G8,mSTZ STZ_estrogen G5,mSTZ STZ_insulin G3,mSTZ control G1
E endo.,7748,NaN,False,0,0,0,0,0,0,0,...,914,3740,2546,1,2,2,1,1,0,2
E non-endo.,29177,NaN,False,1,0,0,0,1,1,1,...,4109,6126,8454,5,2,12,1,1,0,0
alpha,40935,377.0,False,104,302,629,949,308,172,146,...,0,8,21,2895,1487,2539,1052,1441,1099,1141
beta,102143,2782.0,False,4252,3245,4325,1916,445,306,783,...,1,5,17,1496,1135,1670,1421,1608,1204,5795
delta,24775,NaN,False,108,116,254,185,125,74,104,...,1,29,73,1426,1195,1472,787,1349,688,660
gamma,6999,NaN,False,116,251,2,6,9,4,2,...,0,1,5,312,447,341,195,767,267,113
endo. prolif.,887,NaN,False,3,7,15,8,4,1,0,...,2,11,1,11,8,8,15,11,2,17
acinar,480,NaN,False,0,0,0,0,0,0,0,...,0,0,0,1,22,1,0,8,0,0
ductal,8742,NaN,False,0,1,3,5,26,28,9,...,0,2,1,103,42,24,14,61,3,6
endothelial,13469,NaN,False,0,58,5,55,9,2,29,...,0,0,0,443,294,284,71,317,60,240


In [66]:
# Count coarse beta cell subtypes per sample
ct_b_coarse_count=pd.crosstab(
    obs_b['leiden_r1.5_parsed'],
    obs_b['study_parsed_design_sample'])
ct_b_coarse_count.index.name=None
ct_b_coarse_count

study_parsed_design_sample,4m head_Fltp+ mouse2,4m head_Fltp- mouse1,4m tail_Fltp+ mouse4,4m tail_Fltp- mouse3,5wNOD IRE1alphabeta-/- SRR10985097,5wNOD IRE1alphabeta-/- SRR10985098,5wNOD IRE1alphafl/fl SRR10985099,8-16wNOD 14w SRR7610298,8-16wNOD 14w SRR7610299,8-16wNOD 14w SRR7610300,...,embryonic E13.5 E13_5,embryonic E14.5 E14_5,embryonic E15.5 E15_5,mSTZ STZ G2,mSTZ STZ_GLP-1 G4,mSTZ STZ_GLP-1_estrogen G6,mSTZ STZ_GLP-1_estrogen+insulin G8,mSTZ STZ_estrogen G5,mSTZ STZ_insulin G3,mSTZ control G1
imm.,3895,2897,3769,1802,383,279,675,25,8,6,...,0,0,0,18,1,73,8,6,40,190
adult,54,38,73,9,9,1,14,25,23,28,...,0,0,1,63,15,163,426,44,553,4543
agedF,42,24,21,2,7,4,17,0,0,0,...,0,0,1,0,0,2,2,0,7,7
agedM,143,90,96,50,17,8,3,0,1,0,...,0,0,0,1,0,3,11,0,74,132
NOD-D,3,13,28,19,1,4,16,696,554,158,...,0,0,0,8,5,20,20,5,9,32
db/db+mSTZ,48,84,102,12,16,2,5,28,25,15,...,0,4,4,1314,978,1215,843,1459,458,464
chem,32,66,217,9,11,8,50,19,7,25,...,1,1,10,47,36,136,56,45,35,353
lowQ,1,0,4,2,0,0,0,2,1,0,...,0,0,1,33,79,50,24,32,15,50
lowQ-hMT,34,33,15,11,1,0,3,0,1,1,...,0,0,0,12,21,8,31,17,13,24


In [67]:
# Count fine beta cell subtypes per sample
ct_b_fine_count=pd.crosstab(
    obs_b['hc_gene_programs_parsed'],
    obs_b['study_parsed_design_sample'])
ct_b_fine_count.index.name=None
ct_b_fine_count

study_parsed_design_sample,4m head_Fltp+ mouse2,4m head_Fltp- mouse1,4m tail_Fltp+ mouse4,4m tail_Fltp- mouse3,5wNOD IRE1alphabeta-/- SRR10985097,5wNOD IRE1alphabeta-/- SRR10985098,5wNOD IRE1alphafl/fl SRR10985099,8-16wNOD 14w SRR7610298,8-16wNOD 14w SRR7610299,8-16wNOD 14w SRR7610300,...,embryonic E13.5 E13_5,embryonic E14.5 E14_5,embryonic E15.5 E15_5,mSTZ STZ G2,mSTZ STZ_GLP-1 G4,mSTZ STZ_GLP-1_estrogen G6,mSTZ STZ_GLP-1_estrogen+insulin G8,mSTZ STZ_estrogen G5,mSTZ STZ_insulin G3,mSTZ control G1
imm.1,412,602,563,52,7,6,25,17,1,5,...,0,0,0,8,5,33,6,9,11,76
imm.2,890,701,899,937,135,104,185,13,10,1,...,0,0,0,1,0,2,2,0,0,1
imm.3,15,26,10,2,0,1,1,4,1,1,...,0,2,3,45,48,93,21,64,25,92
NOD-imm.,38,31,55,6,202,139,382,0,0,1,...,0,0,0,1,0,1,1,0,0,2
adult-imm.1,2304,1431,2023,797,37,24,67,2,0,0,...,0,0,0,5,1,31,4,3,14,33
adult-imm.2,75,26,71,6,1,0,4,1,0,0,...,0,0,0,1,0,4,14,2,43,19
adult1,185,122,93,22,10,9,19,0,0,0,...,0,0,0,1,0,9,13,1,57,85
adult2,52,34,76,6,4,0,13,20,15,17,...,0,0,0,42,14,93,306,20,370,3921
adult+agedM,79,58,73,37,7,1,2,1,0,0,...,0,0,0,0,0,5,3,0,42,165
agedF1,44,25,16,3,6,3,12,0,0,0,...,0,0,1,0,0,2,3,0,5,9


In [68]:
# Save count data
writer = pd.ExcelWriter(path_tables+'cell_counts.xlsx',engine='xlsxwriter') 
for sheet,count in [
    ('cell_type',ct_count),
    ('beta_subtype_coarse',ct_b_coarse_count),
    ('beta_subtype_fine',ct_b_fine_count)
]:
    count.to_excel(writer, sheet_name=sheet,index=True)   
writer.save()

In [64]:
path_tables+'cell_counts.xlsx'

'/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/tables/paper/cell_counts.xlsx'